# Adventures In In-Flight Entertainment
I bought a subscription to Interview Cake. So far, the questions have been a really good combination of breadth and depth, and today I went pretty far down a rabbit hole with one of the questions. [The question](https://www.interviewcake.com/question/python/inflight-entertainment) is about finding if a list comtains a pair of elements that sum to a specified value. The question goes like this: 

### You've built an inflight entertainment system with on-demand movie streaming.
Users on longer flights like to start a second movie right when their first one ends, but they complain that the plane usually lands before they can see the ending. **So you're building a feature for choosing two movies whose total runtimes will equal the exact flight length.**

Write a function that takes an integer `flight_length` (in minutes) and a list of integers `movie_lengths` (in minutes) and returns a boolean indicating whether there are two numbers in `movie_lengths` whose sum equals `flight_length`.

When building your function:
* Assume your users will watch exactly two movies
* Don't make your users watch the same movie twice
* Optimize for runtime over memory

In [1]:
# My first solution was pretty straight forward. 
def movie_match(flight_length, movie_lengths):
    watched_movies = set()
    for movie_length in movie_lengths:
        complement = flight_length - movie_length
        if complement in watched_movies:
            return True
        watched_movies.add(movie_length)
    return False

I liked this solution. It solves the question in O(n) time, where n is the number of movies, but also terminates early so that's only in the worst case. It also uses O(n) additional memory, but since the question asks us to optimize for runtime over memory, this seems fine. My solution also accounts quite nicely for edge cases:
* If no two movies add up to the given flight time, it runs the entire loop and returns False
* If the first two movies in a list equal the flight time, it returns without finishing the loop.
* If there are multiple movies with the same runtime, we only store them once because we're using a set (as opposed to a list or other non-duplicate data structure)
* We also use a set instead of a hash map because we don't need to store a (key, value), just a key
* If there's a movie with length `flight_length/2`, we don't return a false positive because, we only add a movie to the set *after* we've checked for the complement.

So I was ready to check this quesiton off the list and go on my merry way, but it turns out there were some pretty interesting bonus questions...

### What if we wanted the movie lengths to sum to something close to the flight length (say, within 20 minutes)?
The naive solution here is simply to just place our complement check in a for loop that iterates from `range(movie_length - margin_of_error, movie_length + margin_of_error + 1)`. However, this brings our nice O(n) time complexity to O(n\*m) where me is the margin of error. We can do better.

For my solution, I leveraged the fact that numbers in python can be arbitrarily large, so I can basically create one giant bit vector, then use a mask to check an entire range of times in O(1) time.

In [2]:
def movie_almost_match(flight_length, movie_lengths, margin_of_error):
    wiggle_room = 2 ** (margin_of_error + 1) - 1
    total_range = 0
    for movie_length in movie_lengths:
        high_margin = wiggle_room << (movie_length + 1)
        left_shift = movie_length - margin_of_error - 1
        low_margin = wiggle_room << left_shift if left_shift > 0 else wiggle_room >> abs(left_shift)
        exact_time = 1 << movie_length
        acceptable_range = low_margin | exact_time | high_margin
        if total_range & acceptable_range:
            return True
        complement = flight_length - movie_length
        if complement > 0:
            total_range |= 1 << (flight_length - movie_length)
    return False    

I really like this solution because not only does it not take a hit on runtime, but it will also probably use a bit less memory. In my earlier solution, we had to store n integers, plus the overhead of a set. With this solution, the space complexity becomes on the order of the length of the longest movie, which heuristically-speaking, is max 200 minutes. Especially as your list of movies becomes arbitrarily large, the savings of this over storing a bunch of ints in a set could be significant.

...But wait, there's more!

### What if we wanted to fill the flight length as nicely as possible with any number of movies (not just 2)?
Again, a straight-forward solution is recursive backtracking with a healthy dose of memoization, but I wanted to do one better. This question immediately struck me as a dynamic programming question, and since this is a particularly weak area for me, I went head first.

The spark of intuition came when I realized the similarity between this problem and another Interview Cake question, [Making Change](https://www.interviewcake.com/question/python/coin). The problem here, however, was that just doing this approach verbatum wasn't going to work--it allows you to watch the same movie multiple times. For example, say we have a 100 minute flight and a movie that's 25 minutes long, using the dynamic approach from the earlier question, watching the movie 4 times in a row was considered an acceptable solution. However, the problem scope specificaly states that's not ok. 

Overwriting the actual buffer works up until 2(movie_length) - 1, at which point you start using the current movie's values with itself. My solution? A temp buffer! Instead of writing `buffer[minute] += buffer[minute - movie_length]`, I wrote it as `temp_buffer = buffer[minute] + buffer[minute - movie_length]`. Then, just update the pointer reference and create a new temp buffer (O(n), but that gets eaten by the larger time complexity), and voila! A dynamic approach.

In [3]:
def dynamic_movie_match(flight_length, movie_lengths):
    final_buffer = get_empty_buffer(flight_length)
    temp_buffer = get_empty_buffer(flight_length)
    for movie_length in movie_lengths:
        for minute in range(movie_length, flight_length + 1):
            temp_buffer[minute] = final_buffer[minute] + final_buffer[minute - movie_length]
        final_buffer = temp_buffer
        temp_buffer = get_empty_buffer(flight_length)
        # Quick check for early termination
        if final_buffer[flight_length] > 0:
            return True
    return False

def get_empty_buffer(size):
    return [1] + [0] * size

I'm really freaking proud of this code for a couple reasons:
1. Dynamic programming is really hard for me, and I love how beautifully this code solves the problem! It has O(n\*m) time and O(n) space complexity, which is so much better than the brute force solution, which could be (in the worst case w/o memoization) on the order of n!
2. It's also really easy to adapt this code if the question became "What's the closest time we could get to the entire flight?". Instead of return True/False, you could just return the flight_time, or the closest index to flight_time with a non-zero value, which is pretty neat if I say so myself.
3. It's equally easy to return the number of movies you have to watch to fill your flight time!

Finally, there was one more bonus question:

### What if we knew that movie_lengths was sorted? Could we save some space and/or time?
If we know the array is sorted, then finding complement gets even easier. For finding a given complement, you can walk through the list, and instead of adding the complement to a set, simply binary search for it. This shaves our O(n) space complexity down to O(1), but it does bring the worst case time complexity to O(nlogn). It's hard to get rid of this entirely, but there are a couple optimizations I've considered that can help in the average case.

1. If you have have a 40 minute movie on a 120 minute flight, you do binary search for 80 (40's complement). If you don't find it, you know to move on to the next value. But what if your movies look something like this: `[5, 25, 40, 90, 110]`. You've just compared 40, and your next element is 90. However, the complement of 90 is 30, which is less than 40. Since we've been processing our elements in sorted order, we know 30 isn't in the list because we would've processed it before 40. By keeping track of the smallest complement we've seen so far, we can stop our search early if we know we won't find anything in the other half. This doesn't help with big O complexity because in the worst case, no two pairs in the list are big enough to sum to your flight time, so you're constantly doing a binary search for something larger than the max value of the list. However, it could definitely be a time saver in the average case when n is really large.
2. The max of your list will always be the last element in a sorted list. If the complement you're looking for is greater than the max of your list, you can skip the binary search. The logn time it takes for a binary search isnt' that big of a deal, but it still prevents unnecessary calls.
3. Conversely, min of your list will be the first element. If the complement you're looking for is smaller than the min of your list, you can actually just stop looking because each successive element is only gonna get bigger and bigger.

So with these ideas in mind, here's my code for the final part of the bonus question:

In [4]:
import math

def movie_match_sorted(flight_length, movie_lengths):
    min_length = movie_lengths[0]
    max_length = movie_lengths[-1]
    min_complement = math.inf
    for movie_length in movie_lengths:
        complement = flight_length - movie_length
        if movie_length > min_complement or complement < min_length:
            return False
        complement_in_list = False if complement > max_length else binary_search(movie_lengths, complement)
        if complement_in_list:
            return True
        
    return False

def binary_search(sorted_list, el):
    start = 0
    end = len(sorted_list)
    while start <= end:
        middle = (start + end) // 2
        if sorted_list[middle] < el:
            start = middle + 1
        elif sorted_list[middle] > el:
            end = middle - 1
        else:
            return True    

# Tests
Finally, the time has come for some tests! The testing below isn't the most beautiful ever, but it works enough to check my sanity.

In [5]:
tests = [(100, [15, 25, 30, 60, 90]), (100, [10, 25, 30, 60, 80]), (100, [25, 25, 25, 25]), (100, [25, 25, 30]), \
        (100, [10, 20, 30])]

for test in tests:
    print("Testing:", test)
    
    print("   These should have the same result")
    print("     ", movie_match(*test))
    print("     ", movie_match_sorted(*test))
    print("   These can have different answers.")
    print("     With margin of error:", movie_almost_match(*test, 10))
    print("     With more than 2 movies:", dynamic_movie_match(*test))
    print('')


Testing: (100, [15, 25, 30, 60, 90])
   These should have the same result
      False
      False
   These can have different answers.
     With margin of error: True
     With more than 2 movies: True

Testing: (100, [10, 25, 30, 60, 80])
   These should have the same result
      False
      False
   These can have different answers.
     With margin of error: True
     With more than 2 movies: False

Testing: (100, [25, 25, 25, 25])
   These should have the same result
      False
      False
   These can have different answers.
     With margin of error: False
     With more than 2 movies: True

Testing: (100, [25, 25, 30])
   These should have the same result
      False
      False
   These can have different answers.
     With margin of error: False
     With more than 2 movies: False

Testing: (100, [10, 20, 30])
   These should have the same result
      False
      False
   These can have different answers.
     With margin of error: False
     With more than 2 movies: False
